In [9]:
from Algorithms.clarke_wright import ClarkeWright
from Algorithms.client import Client
import pandas as pd
from datetime import datetime, timedelta
import random
import names
import json
import numpy as np

# Generate historized client data

In [10]:

def generate_client_list(num_clients=50, start_date="2024-01-01", end_date="2024-12-31", 
                         min_slots=1, max_slots=3, locations=None):
    if locations is None:
        locations = ['Geldrop', 'Helmond', 'Someren', 'Deurne Vlierden', 'Mierlo']
    
    date_format = "%Y-%m-%d"
    start_date = datetime.strptime(start_date, date_format)
    end_date = datetime.strptime(end_date, date_format)
    date_range = (end_date - start_date).days

    timeslots = ['morning', 'evening']
    clients = []

    for _ in range(num_clients):
        name = names.get_full_name()  # Generating random full names
        location = random.choice(locations)
        
        # Generate appointment day
        appointment_day = start_date + timedelta(days=random.randint(0, date_range))
        appointment_hour = random.randint(0, 23)
        appointment_minute = random.randint(0, 59)
        appointment_time = datetime.combine(appointment_day, datetime.min.time()) + timedelta(hours=appointment_hour, minutes=appointment_minute)
        appointment_time_str = appointment_time.strftime(f"{date_format} %H:%M:%S")

        num_slots = random.randint(min_slots, max_slots)
        availability = []

        # Generate unique availability slots within 2 weeks after the appointment day or until end_date
        for _ in range(num_slots):
            while True:
                max_avail_day = min(appointment_day + timedelta(days=14), end_date)
                if max_avail_day <= appointment_day:
                    break
                day = appointment_day + timedelta(days=random.randint(1, (max_avail_day - appointment_day).days))
                timeslot = random.choice(timeslots)
                slot = f"{day.strftime(date_format)}_{timeslot}"

                # Ensure the new slot is unique
                if slot not in availability:
                    availability.append(slot)
                    break
        
        client = Client(name, location, availability, appointment_time_str)
        clients.append(client)

    return clients


config = {
    'num_clients': 50,
    'start_date': "2024-06-01",
    'end_date': "2024-06-07",
    'min_slots': 2,
    'max_slots': 2,
    'locations': ['Asten Heusden Ommel', 'Deurne Vlierden', 'Geldrop', 'Gemert Handel', 'Helmond',
                  'Helmond Brandevoort', 'Mierlo', 'Nuenen Gerwen Nederwetten', 'Someren'],
    'distance_matrix_path': "..//Data//distance_matrix.csv",
    'batch_size': 7  # For offline use case, processing batches of clients every 7 days
}

# random seed
random.seed(42)
 
# Example usage:
clients = generate_client_list(config['num_clients'], config['start_date'], config['end_date'], config['min_slots'],
                               config['max_slots'], config['locations'])
for i, client in enumerate(clients):
    print(f"Client {i + 1}: {client.name}, {client.location}, {client.availability}, {client.appointment_time}")

Client 1: James Lawrence, Gemert Handel, ['2024-06-07_evening', '2024-06-03_morning'], 2024-06-02 23:06:00
Client 2: Charles Whaley, Asten Heusden Ommel, ['2024-06-06_evening', '2024-06-07_morning'], 2024-06-05 06:45:00
Client 3: Travis Bowen, Geldrop, ['2024-06-06_morning', '2024-06-05_evening'], 2024-06-02 10:06:00
Client 4: Meghan Sartin, Someren, ['2024-06-07_evening', '2024-06-06_morning'], 2024-06-01 12:05:00
Client 5: John Murray, Helmond, ['2024-06-05_evening', '2024-06-03_evening'], 2024-06-01 07:55:00
Client 6: Kimberly Snyder, Deurne Vlierden, ['2024-06-06_evening', '2024-06-07_evening'], 2024-06-05 20:10:00
Client 7: Curtis Books, Asten Heusden Ommel, ['2024-06-06_evening', '2024-06-03_morning'], 2024-06-02 01:51:00
Client 8: Deborah Ponce, Nuenen Gerwen Nederwetten, ['2024-06-07_evening', '2024-06-05_morning'], 2024-06-02 08:08:00
Client 9: Carl Robinson, Asten Heusden Ommel, [], 2024-06-07 03:09:00
Client 10: Cathy Lamb, Nuenen Gerwen Nederwetten, ['2024-06-06_evening', '

# test online case

In [89]:
def get_definitive_timeslot_clarke(selected_slots, user_data, appointments_data, distance_matrix_path):
    """
    :param selected_slots:
    :param user_data:
    :param appointments_data: List of current appointments data
    :param distance_matrix_path: Path to the distance matrix CSV file
    :return: The definitive timeslot based on the selected slots and user data
    """
    appointments_in_slot_new = []
    appointments_in_slot_old = []

    appointments_in_slot_new.append({
        "username": user_data['username'],
        "location": user_data['location'],
        "appointment_date": selected_slots
    })

    solutions_new = []
    solutions_old = []
    print(selected_slots)
    print(appointments_data)
    for slot in selected_slots:
        if appointments_data:
            for row in appointments_data:
                if row['appointment_date'][0] == slot:
                    appointments_in_slot_old.append(row)
                    appointments_in_slot_new.append(row)
                else:
                    appointments_in_slot_old.append({
                        "username": user_data['username'],
                        "location": user_data['location'],
                        "appointment_date": selected_slots
                    })
                    appointments_in_slot_new.append({
                        "username": user_data['username'],
                        "location": user_data['location'],
                        "appointment_date": selected_slots
                    })
        else:
            appointments_in_slot_old.append({
                "username": user_data['username'],
                "location": user_data['location'],
                "appointment_date": selected_slots
            })
            appointments_in_slot_new.append({
                        "username": user_data['username'],
                        "location": user_data['location'],
                        "appointment_date": selected_slots
             })
            


        print(appointments_in_slot_old)
        clients_old = [Client(appointment['username'], appointment['location'], appointment['appointment_date']) for appointment in appointments_in_slot_old]
        print(f"Slot: {slot}")
        for clients in clients_old:
            print(clients.name, clients.location, clients.availability, clients.appointment_time)
        clarke_old = ClarkeWright(clients_old)
        clarke_old.solve(slot, distance_matrix_path)
        solutions_old.append([clarke_old.get_solution(), slot])
    
        clients_new = [Client(appointment['username'], appointment['location'], appointment['appointment_date']) for appointment in appointments_in_slot_new]
        clarke_new = ClarkeWright(clients_new)
        clarke_new.solve(slot, distance_matrix_path)
        solutions_new.append([clarke_new.get_solution(), slot])

    smallest_time_increase_route = []
    time_increase = np.inf
    for i in range(len(solutions_old)):
        if solutions_new[i][0][1] - solutions_old[i][0][1] < time_increase:
            time_increase = solutions_new[i][0][1] - solutions_old[i][0][1]
            smallest_time_increase_route = solutions_new[i]

    appointments_data.append({"username": user_data['username'], "location": user_data['location'], "appointment_date": [smallest_time_increase_route[1]]})

    return f"Appointment made at slot: {smallest_time_increase_route[1]}", appointments_data


def online_use_case(clients, distance_matrix_path):
    appointments_data = []

    for client in sorted(clients, key=lambda x: x.appointment_time):
        selected_slots = client.availability
        user_data = {
            "username": client.name,
            "location": client.location
        }
        print(f"Client {clients.index(client)}: {client.name}, {client.location}, {client.availability}, {client.appointment_time}")
        message, appointments_data = get_definitive_timeslot_clarke(selected_slots, user_data, appointments_data, distance_matrix_path) 
        # print client num and message
        print(f"Client {clients.index(client)}: {message} \n")

In [90]:
online_use_case(clients, config['distance_matrix_path'])

Client 18: Wade White, Someren, ['2024-06-02_morning', '2024-06-03_evening'], 2024-06-01 01:41:00
['2024-06-02_morning', '2024-06-03_evening']
[]
[{'username': 'Wade White', 'location': 'Someren', 'appointment_date': ['2024-06-02_morning', '2024-06-03_evening']}]
Slot: 2024-06-02_morning
Wade White Someren ['2024-06-02_morning', '2024-06-03_evening'] None
[{'username': 'Wade White', 'location': 'Someren', 'appointment_date': ['2024-06-02_morning', '2024-06-03_evening']}, {'username': 'Wade White', 'location': 'Someren', 'appointment_date': ['2024-06-02_morning', '2024-06-03_evening']}]
Slot: 2024-06-03_evening
Wade White Someren ['2024-06-02_morning', '2024-06-03_evening'] None
Wade White Someren ['2024-06-02_morning', '2024-06-03_evening'] None


IndexError: list index out of range

# test offline case

In [ ]:
def offline_use_case(clients, batch_size, distance_matrix_path):
    start_date = min(client.appointment_time for client in clients)
    end_date = max(client.appointment_time for client in clients)
    current_date = start_date

    appointments_data = []

    while current_date <= end_date:
        batch_clients = [client for client in clients if current_date <= client.appointment_time < current_date + timedelta(days=batch_size)]

        if batch_clients:
            for client in batch_clients:
                client.set_scheduled(current_date.strftime("%Y-%m-%d"))

            clarke = ClarkeWright(batch_clients)
            for client in batch_clients:
                selected_slots = client.availability
                user_data = {
                    "username": client.name,
                    "location": client.location
                }
                message, appointments_data = get_definitive_timeslot_clarke(selected_slots, user_data, appointments_data, distance_matrix_path)
                print(message)

        current_date += timedelta(days=batch_size)

In [ ]:
print("\nOffline Use Case Results:")
offline_use_case(clients, config['batch_size'], config['distance_matrix_path'])